# Modelo no supervisado de clasificación de texto

## Introducción

En este notebook se presenta un modelo no supervisado de clasificación de texto. El modelo se basa en el uso de embeddings de palabras y clustering. Se utiliza el algoritmo de clustering KMeans para agrupar los textos en clusters. 



In [9]:
### Librerias necesarias Doc2Vec

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

### Librerias necesarias para el preprocesamiento de texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

### Descargar stopwords
nltk.download('stopwords')
nltk.download('punkt')


### Otras librerias necesarias
import numpy as np
import pandas as pd
import re
import os

[nltk_data] Downloading package stopwords to /home/isaac-
[nltk_data]     zainea/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/isaac-
[nltk_data]     zainea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.1 Modelo Doc2Vec

El modelo Doc2Vec es una extensión del modelo Word2Vec que agrega un vector adicional para cada documento en el corpus. Este vector adicional se entrena junto con los vectores de palabras y se utiliza para representar el contenido de un documento en un espacio vectorial. Esto permite comparar documentos y encontrar similitudes entre ellos. En este caso ocurre lo siguiente:

#### ¿Cómo funciona Doc2Vec?

Doc2Vec tiene dos enfoques principales:

1. **PV-DM (Distributed Memory):** Este modelo predice una palabra en el documento usando el contexto de palabras a su alrededor y un vector que representa al documento completo. En otras palabras:
   - Se toma un documento y se le asigna un vector único (el vector del documento).
   - Luego, junto con las palabras cercanas, este vector se usa para predecir una palabra en el documento.
   - Al entrenar el modelo, este vector se va ajustando para que capture mejor el significado del documento.

2. **PV-DBOW (Distributed Bag of Words):** Aquí se usa solo el vector del documento para predecir palabras al azar dentro del documento, sin tener en cuenta el contexto de palabras. Es más rápido, pero menos preciso.

En resumen, Doc2Vec es un modelo que entrena vectores para palabras y documentos al mismo tiempo. Estos vectores pueden ser usados luego para comparar documentos, encontrar similitudes entre ellos, o incluso para clasificarlos.

![Proceso doc2vec](../../Datos/Imágenes/doc2vec.png)


## 2. Cargar el texto

El dataset "es_tweets_laboral" de la colección "somosnlp-hackathon-2022" en Hugging Face está diseñado específicamente para el análisis de texto relacionado con temas laborales en español. Este dataset contiene tuits que abordan temas laborales, y es ideal para tareas de clasificación de texto, análisis de sentimientos, y otras aplicaciones de procesamiento de lenguaje natural (NLP) enfocadas en el ámbito laboral.

**Características del Dataset:**
- *Contenido*: Incluye tuits en español relacionados con temas laborales, como empleo, condiciones de trabajo, y derechos laborales.
- *Etiquetas*: Los tuits pueden estar etiquetados según el tema o el sentimiento, lo que facilita su uso en tareas de clasificación supervisada.
- *Aplicaciones*: Este dataset es útil para construir modelos que analicen la percepción de los usuarios sobre temas laborales, detectar tendencias en el mercado laboral, o identificar problemas comunes en el ámbito laboral.

El dataset es parte de un esfuerzo colaborativo durante el Hackathon de SomosNLP en 2022, que busca fomentar el desarrollo de tecnologías de procesamiento de lenguaje natural en español.



In [10]:
### Carga dataset desde huggingface

from datasets import load_dataset
import tqdm as notebook_tqdm
import re

# Cargando el dataset "es_tweets_laboral" desde Hugging Face
dataset = load_dataset("somosnlp-hackathon-2022/es_tweets_laboral")

# Explorando el contenido del dataset
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'entities'],
        num_rows: 184
    })
    test: Dataset({
        features: ['text', 'intent', 'entities'],
        num_rows: 47
    })
})


In [11]:
train = dataset['train']
test = dataset['test']

# Explorando el contenido de los datos de entrenamiento

train_df = train.to_pandas()
test_df = test.to_pandas()
print("Shape of train data: ", train_df.shape)
print("Shape of test data: ", test_df.shape)

train_df.head()

Shape of train data:  (184, 3)
Shape of test data:  (47, 3)


,text,intent,entities
0,"ni siquiera decir ""palabras sucias"" te hace me...",abuso_autoridad,"[{'value': 'mal jefe,', 'entity': 'denuncia'}]"
1,"un amigo de mi hermana le comentó, dudo mucho ...",salario_precario,"[{'value': 'nos maltratan con el salario', 'en..."
2,respecto a los salarios: todos merecemos un sa...,salario_precario,[{'value': 'salario que nos permita vivir dign...
3,@FOVISSSTEmx @fovissste Gracias por su atenció...,derechos_laborales,[{'value': 'no es un esquema que favorezca al ...
4,"docentes venezolanos, perciben salarios insufi...",salario_precario,"[{'value': 'perciben salarios insuficientes', ..."


In [12]:
test_df.head()

,text,intent,entities
0,#OficinaDeEnvigado #TrabajoPorHoras #AbusoLabo...,explotacion_laboral,"[{'value': '#AbusoLaboral', 'entity': 'denunci..."
1,@MarianaLaActriz Oiga no la he visto cotizar e...,acoso_laboral,"[{'value': 'acoso sexual', 'entity': 'denuncia..."
2,La Asociación de Rectores Universitarios exigi...,salario_precario,"[{'value': 'ajuste del incremento salarial', '..."
3,"15).-Venezuela. Gobierno. Rusia. Presenta, en ...",salario_precario,[{'value': 'índice relativamente bajo de desem...
4,Después que Maduro bajó el salario mínimo de $...,salario_precario,"[{'value': 'bajó el salario mínimo', 'entity':..."


## 3. Preprocesamiento

Empezamos por cargar el dataset y realizar un preprocesamiento básico de los textos. En este caso, se eliminan las menciones a usuarios, los enlaces, y los caracteres especiales. Además, se convierten los textos a minúsculas y se eliminan las stopwords.

In [13]:
########## Preprocesamiento de texto ###############

def preprocess_text(text):
    # Eliminando caracteres especiales y números
    text = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚ\s]', '', text, re.I|re.A)
    # Convertir a minúsculas
    text = text.lower()
    # eliminando stopwords
    stop_words = set(stopwords.words('spanish'))
    word_tokens = word_tokenize(text)
    text = [i for i in word_tokens if not i in stop_words]
    text = ' '.join(text)
    return text

# Aplicando la función de preprocesamiento a los datos de entrenamiento y prueba

train_df['text_pre'] = train_df['text'].apply(preprocess_text)

test_df['text_pre'] = test_df['text'].apply(preprocess_text)

train_df.head()

,text,intent,entities,text_pre
0,"ni siquiera decir ""palabras sucias"" te hace me...",abuso_autoridad,"[{'value': 'mal jefe,', 'entity': 'denuncia'}]",siquiera decir palabras sucias hace menos prof...
1,"un amigo de mi hermana le comentó, dudo mucho ...",salario_precario,"[{'value': 'nos maltratan con el salario', 'en...",amigo hermana comentó dudo vuelva trabajar uni...
2,respecto a los salarios: todos merecemos un sa...,salario_precario,[{'value': 'salario que nos permita vivir dign...,respecto salarios merecemos salario permita vi...
3,@FOVISSSTEmx @fovissste Gracias por su atenció...,derechos_laborales,[{'value': 'no es un esquema que favorezca al ...,fovissstemx fovissste gracias atención desgrac...
4,"docentes venezolanos, perciben salarios insufi...",salario_precario,"[{'value': 'perciben salarios insuficientes', ...",docentes venezolanos perciben salarios insufic...


## 4. Modelo de Clasificación de Texto

Una vez que hemos preprocesado los textos, podemos aplicar un modelo de clasificación no supervisado para agruparlos en categorías o clusters. En este caso, utilizaremos el algoritmo de clustering KMeans para agrupar los textos en clusters. AUnque primero debemos convertir los textos en vectores numéricos utilizando embeddings de palabras.


In [14]:
################## Entrenamiento de Word2Vec ####################

# Tokenizando el texto

train_df['text_tokens'] = train_df['text_pre'].apply(lambda x: x.split())
test_df['text_tokens'] = test_df['text_pre'].apply(lambda x: x.split())

train_df.head()

,text,intent,entities,text_pre,text_tokens
0,"ni siquiera decir ""palabras sucias"" te hace me...",abuso_autoridad,"[{'value': 'mal jefe,', 'entity': 'denuncia'}]",siquiera decir palabras sucias hace menos prof...,"[siquiera, decir, palabras, sucias, hace, meno..."
1,"un amigo de mi hermana le comentó, dudo mucho ...",salario_precario,"[{'value': 'nos maltratan con el salario', 'en...",amigo hermana comentó dudo vuelva trabajar uni...,"[amigo, hermana, comentó, dudo, vuelva, trabaj..."
2,respecto a los salarios: todos merecemos un sa...,salario_precario,[{'value': 'salario que nos permita vivir dign...,respecto salarios merecemos salario permita vi...,"[respecto, salarios, merecemos, salario, permi..."
3,@FOVISSSTEmx @fovissste Gracias por su atenció...,derechos_laborales,[{'value': 'no es un esquema que favorezca al ...,fovissstemx fovissste gracias atención desgrac...,"[fovissstemx, fovissste, gracias, atención, de..."
4,"docentes venezolanos, perciben salarios insufi...",salario_precario,"[{'value': 'perciben salarios insuficientes', ...",docentes venezolanos perciben salarios insufic...,"[docentes, venezolanos, perciben, salarios, in..."


In [15]:
# Entrenando el modelo Doc2Vec

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_df['text_tokens'])]

# Definiendo el modelo Doc2Vec

model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

# Guardando el modelo entrenado

model.save("doc2vec.model")


In [18]:
# Cargando el modelo entrenado

model = Doc2Vec.load("doc2vec.model")

# Obteniendo el vector de una palabra

model.wv['trabajo']

# Obteniendo las palabras más similares a una palabra

model.wv.most_similar('abuso')


[('tv', 0.3388918340206146),
 ('páginas', 0.33612701296806335),
 ('inspección', 0.32902324199676514),
 ('casa', 0.30751997232437134),
 ('gómezpalacios', 0.3025720715522766),
 ('interiorizado', 0.3020995855331421),
 ('fernand', 0.28665125370025635),
 ('cuantamugre', 0.28345754742622375),
 ('terminó', 0.27055153250694275),
 ('maldonado', 0.2618955671787262)]

In [19]:
# Obteniendo la similitud entre dos palabras

model.wv.similarity('trabajo', 'empleo')


0.10175322

In [20]:
#### Hacemos un clustering de los tweets con KMeans

from sklearn.cluster import KMeans

# Obteniendo los vectores de los tweets

vectors = [model.infer_vector(doc) for doc in train_df['text_tokens']]

# Definiendo el modelo KMeans

kmeans = KMeans(n_clusters=5, random_state=0)

# Entrenando el modelo KMeans

kmeans.fit(vectors)

# Obteniendo las etiquetas de los clusters

train_df['cluster'] = kmeans.labels_

# Explorando los clusters

train_df['cluster'].value_counts()



cluster
3    55
4    37
0    37
2    29
1    26
Name: count, dtype: int64

In [21]:
### Veamos los tweets de un cluster

pd.set_option('display.max_colwidth', None)

train_df[train_df['cluster'] == 0]['text'].head(10)

5                                                                                                                            es un poco injusto que haya gente que batalla para tener un buen salario o trabajo y que haya gente que esté bien acomodada y viviendo bien sin necesidad de estar batallando
8                                                                                                                                                                                 Dos profesoras e investigadoras de la Universidad del Istmo #Unistmo, campus Ixtepec, denunciaron violencia laboral y h…
10                                                                                                                                                                       los #bomberosforestales siguen su lucha! por unas #condicioneslaborales dignas. por no tener que lamentarse tarde. soluciones ya!
13                                                                       @CarolinaPiparo Carolina...los

In [22]:
### Veamos el centroide de cada cluster

for i in range(5):
    print("Cluster ", i)
    print(model.wv.most_similar(positive=[kmeans.cluster_centers_[i]], topn=10))
    print("\n\n")

Cluster  0
[('tv', 0.3773950934410095), ('morena', 0.35579192638397217), ('salario', 0.3412707448005676), ('medida', 0.32781466841697693), ('quedar', 0.32356029748916626), ('hoy', 0.2996211349964142), ('salum', 0.29874157905578613), ('si', 0.2883167862892151), ('superintendencia', 0.28237178921699524), ('recae', 0.28189268708229065)]



Cluster  1
[('si', 0.31819480657577515), ('presidenta', 0.303609699010849), ('ls', 0.3002225160598755), ('planteamos', 0.29824796319007874), ('ahorra', 0.2975274622440338), ('sacando', 0.2972254753112793), ('necesitan', 0.29635271430015564), ('salum', 0.284090131521225), ('locales', 0.2769164443016052), ('salario', 0.2698799669742584)]



Cluster  2
[('vida', 0.34470483660697937), ('q', 0.3345752954483032), ('trabajo', 0.32723310589790344), ('región', 0.31122347712516785), ('todas', 0.300120085477829), ('firmes', 0.2965523898601532), ('misoginia', 0.2937416136264801), ('diciendo', 0.28792816400527954), ('universidad', 0.286060094833374), ('importante', 